In [1]:
import pandas as pd
import numpy as np

def wraptable(thing, caption):
    # replace ascii bs
    thing = thing.replace("\\textasciicircum \\{a\\}", "\\textsuperscript{a}").replace("\\textasciicircum \\{b\\}", "\\textsuperscript{b}" )
    for substr in ['MUTAG', 'ER\\textsuperscript{a}', 'BA\\textsuperscript{a}', 'HK\\textsuperscript{a}', 'WC\\textsuperscript{a}', 'ForcedRB &']:
        thing = thing.replace(substr, f'\midrule \n{substr}', 1)
    thing = thing.replace("@@@", "")
    thing = thing.replace("Nmin", "$N_\\text{min}$")
    thing = thing.replace("Nmax", "$N_\\text{max}$")
    thing = thing.replace("gurobi", 'Gurobi')
    thing = thing.replace("LiftMP", 'OptGNN')
    replace = {'REDDIT-BINARY': '\midrule \nREDDIT-BIN\\textsuperscript{c}',
           'REDDIT-MULTI-12K': 'REDDIT-M-12K\\textsuperscript{c}',
            'REDDIT-MULTI-5K': 'REDDIT-M-5K\\textsuperscript{c}',
            'gen\\_n': "$N_\\text{min}$",
            'greedy': 'Greedy',
            'Type': 'Dataset',
            'dataset': 'Dataset'} 
    for k, v in replace.items():
        thing = thing.replace(k, v)
    return thing
    #return "\\begin{table} \n" + thing + f"\n\\caption{{ {caption}}} \n" + "\\end{table}\n"


def do_bolding(latex_string, list_of_things):
    dict_of_things = {str(x): f'\\textbf{{{str(x)}}}' for x in list_of_things}
    for k, v in dict_of_things.items():
        latex_string = latex_string.replace(k, v)
    return latex_string
    
    
    

In [2]:
fname = 'Table1_maxcut.csv'
#fname = 'Table1_VC.csv'
df = pd.read_csv(fname)
df = df[~df.dataset.isin(['RANDOM', 'ForcedRB'])]
dataset_conversion = dict(zip(['BarabasiAlbert', 'ErdosRenyi', 'PowerlawCluster', 'WattsStrogatz', 'MUTAG', 'ENZYMES', 'PROTEINS',   'IMDB-BINARY', 'COLLAB',], 
                              ['BA^{a}', 'ER^{a}', 'HK^{a}', 'WC^{a}', 'MUTAG^{b}', 'ENZYMES^{b}', 'PROTEINS^{b}',   'IMDB-BIN^{b}', 'COLLAB^{b}',]))

df.dataset = df.dataset.map(lambda x: dataset_conversion.get(x, x))
#df = pd.read_csv('Table1_VC.csv')

In [3]:
gen_n_dict = dict(zip([50, 100, 400, "", np.nan], [100, 200, 500, "", ""]))
nudf = df.round(2).fillna("")
nudf.gen_n = nudf.gen_n.apply(lambda x: int(x) if x != "" else x)
nudf['Nmax'] = nudf.gen_n.apply(lambda x: gen_n_dict[x])
nudf['Nmin'] = nudf.gen_n
nudf.dataset = nudf.apply(lambda row: f'{row.dataset} ({row.Nmin},{row.Nmax})', axis=1)
nudf.dataset = nudf.dataset.apply(lambda dataset: dataset[:-4] if dataset.endswith("(,)") else dataset)

nudf = nudf[['dataset', 'GAT', 'GCNN', 'GIN', 'GatedGCNN', 'LiftMP']]

# get things to bold
list_of_things = []
for i, row in nudf.iterrows():
    possibilities = [float(x.split(" ")[0]) for x in row.values[2:] if x!=""]
    if fname == 'Table1_maxcut.csv':
        list_of_things.append(f'{max(possibilities):0.2f}')
    else:
        list_of_things.append(f'{min(possibilities):0.2f}')

main_body = (nudf.to_latex(index=False))
print(wraptable(do_bolding(main_body,list_of_things),fname))

\begin{tabular}{llllll}
\toprule
         Dataset &           GAT &          GCNN &           GIN &     GatedGCNN &        OptGNN \\
\midrule
 \midrule 
ER\textsuperscript{a} (50,100) &    525.9 (25) &    500.9 (17) &    498.8 (15) &    526.8 (14) &    529.0 (18) \\
ER\textsuperscript{a} (100,200) &   1979.4 (20) &   1890.1 (23) &   1890.0 (23) &   1978.8 (22) &   1995.1 (24) \\
ER\textsuperscript{a} (400,500) & 16317.7 (208) & 15692.1 (230) & 15818.4 (212) & 16188.8 (211) & 16387.5 (226) \\
       \midrule 
MUTAG\textsuperscript{b} &     27.9 (18) &      9.4 (20) &      9.4 (13) &      9.4 (10) &     27.9 (10) \\
     ENZYMES\textsuperscript{b} &     80.7 (14) &     31.2 (16) &     74.6 (14) &     31.2 (11) &     81.4 (13) \\
    PROTEINS\textsuperscript{b} &    100.9 (14) &     39.1 (13) &     93.0 (12) &     39.1 (17) &    102.2 (16) \\
    IMDB-BIN\textsuperscript{b} &     82.3 (19) &     45.6 (19) &     45.6 (11) &     45.6 (13) &     97.5 (16) \\
      COLLAB\textsuperscript{b} &

/tmp/ipykernel_1782492/2663717676.py:20: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  main_body = (nudf.to_latex(index=False))


In [4]:
nudf.keys()

Index(['dataset', 'GAT', 'GCNN', 'GIN', 'GatedGCNN', 'LiftMP'], dtype='object')

# Table 2

In [5]:
fname = 'Table2_maxcut.csv'
#fname = 'Table2_VC.csv'

df = pd.read_csv(fname)
df = df[~df.Type.isin(['RANDOM', 'ForcedRB'])]

#dataset_conversion = dict(zip(['BarabasiAlbert', 'ErdosRenyi', 'PowerlawCluster', 'WattsStrogatz', 'MUTAG', 'ENZYMES', 'PROTEINS',   'IMDB-BINARY', 'COLLAB',], 
#                              ['BA^{b}', 'ER^{b}', 'PC^{b}', 'WC^{b}', 'MUTAG^{a}', 'ENZYMES^{a}', 'PROTEINS^{a}',   'IMDB-BINARY^{a}', 'COLLAB^{a}',]))

df.Type = df.Type.map(lambda x: dataset_conversion.get(x, x))#df = pd.read_csv('Table2_VC.csv')


df

# load in nikos baselines
if fname == 'Table2_VC.csv':
    x = pd.read_csv('kamis_lwd_baselines.csv')
    problems = {'er': 'ErdosRenyi_400.0',
            'ba': 'BarabasiAlbert_400.0',
            'hk': 'PowerlawCluster_400.0',
            'ws': 'WattsStrogatz_400.0',}

    for i, row in x.iterrows():
        problem = problems[row['problem']]
        df.loc[df['Unnamed: 0'] == problem, 'kamis'] = f'{row["k_solution_sizes_average"]:0.2f} ({row["k_times_average"]:0.2f})'
        df.loc[df['Unnamed: 0'] == problem, 'lwd'] = f'{row["l_solution_sizes_average"]:0.2f} ({row["l_times_average"]:0.2f})'
        
    

In [6]:
nudf = df.iloc[:,1:].round(2).fillna("")
#
#print(nudf.keys())



nudf.Nmin = nudf.Nmin.apply(lambda x: int(x) if x!="" else x)
nudf.Nmax = nudf.Nmax.apply(lambda x: int(x) if x!="" else x)
nudf.Type = nudf.apply(lambda row: f'{row.Type} ({row.Nmin},{row.Nmax})', axis=1)
nudf.Type = nudf.Type.apply(lambda dataset: dataset[:-4] if dataset.endswith("(,)") else dataset)
#nudf['dataset'] = nudf.Type
nudf = nudf[[x for x in nudf.keys() if x not in ['Nmin', 'Nmax', 'SDP proj'
]]]

# TODO: get things to bold
# get things to bold

list_of_things = []
for i, row in nudf.iterrows():
    #print(row)
    possibilities = [float(x.split(" ")[0]) if isinstance(x, str) else x for x in row.values[2:]]
    if fname == 'Table1_maxcut.csv':
        list_of_things.append(f'{max(possibilities):0.2f}')
    else:
        list_of_things.append(f'{min(possibilities):0.2f}')
list_of_things = []

nudf = nudf[['Type', 'LiftMP', 'greedy', 'gurobi_0.1', 'gurobi_1.0', 'gurobi_8.0']]


cols = nudf.columns
hrow1 = []
hrow2 = []
for x in cols:
    if x.startswith('gurobi'):
        hrow1.append("gurobi")
        hrow2.append(x.split("_")[1] + "s")
    else:
        hrow1.append(x)
        hrow2.append("@@@")

header = pd.MultiIndex.from_arrays([hrow1, hrow2])
nudf.columns = header

main_body = nudf.to_latex(index=False)

print(wraptable(do_bolding(main_body,list_of_things),fname))



\begin{tabular}{llrrrr}
\toprule
            Dataset &        OptGNN & Greedy & \multicolumn{3}{l}{Gurobi} \\
              &            &     &    0.1s &    1.0s &    8.0s \\
\midrule
 \midrule 
BA\textsuperscript{a} (50,100) &    351.5 (22) &  200.1 &   351.9 &   352.1 &   352.1 \\
BA\textsuperscript{a} (100,200) &    717.2 (20) &  408.0 &   719.4 &   719.7 &   720.2 \\
BA\textsuperscript{a} (400,500) &   2198.1 (60) & 1255.2 &  2208.1 &  2208.1 &  2212.5 \\
 \midrule 
ER\textsuperscript{a} (50,100) &    529.0 (18) &  298.5 &   529.9 &   530.0 &   530.2 \\
ER\textsuperscript{a} (100,200) &   1995.1 (24) & 1097.3 &  2002.9 &  2002.9 &  2002.9 \\
ER\textsuperscript{a} (400,500) & 16387.5 (226) & 8622.3 & 16476.7 & 16491.6 & 16495.3 \\
 \midrule 
HK\textsuperscript{a} (50,100) &    345.7 (18) &  196.2 &   346.2 &   346.4 &   346.4 \\
HK\textsuperscript{a} (100,200) &    709.4 (20) &  402.5 &   711.7 &   712.3 &   712.9 \\
HK\textsuperscript{a} (400,500) &   2159.9 (60) & 1231.0 &  2169.

/tmp/ipykernel_1782492/1725993717.py:45: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  main_body = nudf.to_latex(index=False)


# instructions: 
2. add midrules